## Exercise 1: Recipe Database

### 1.1 

Load the JSON recipes database provided in the data folder. (It has been zipped to save space.)

How many of the recipes are for breakfast food? 

Hint: The `description` would contain the work "breakfast".

In [ ]:
# exercise 1.1

#recipes = pd.read_json(your_path, compression='infer', lines = True) 
# - please read-up on the params. It will be important for future workshops
#Parameters: 
"""
lines : bool, default False Read the file as a json object per line.
compression: str or dict, default ‘infer’ 
            If ‘infer’ and ‘path_or_buf’ is path-like, 
            then detect compression from the following extensions:
            ‘.gz’, ‘.bz2’, ‘.zip’, ‘.xz’, ‘.zst’, ‘.tar’, ‘.tar.gz’, ‘.tar.xz’ or ‘.tar.bz2’ (otherwise no compression).
            If using ‘zip’ or ‘tar’, the ZIP file must contain only one data file to be read in. 
            Set to None for no decompression. 
            Can also be a dict with key 'method' set to one of {'zip', 'gzip', 'bz2', 'zstd', 'tar'} 
            and other key-value pairs are forwarded to zipfile.ZipFile, gzip.GzipFile, bz2.BZ2File, 
            zstandard.
            ZstdDecompressor or tarfile.TarFile, respectively. 
            As an example, the following could be passed for Zstandard decompression using a custom compression dictionary: 
            compression={'method': 'zstd', 'dict_data': my_compression_dict}.
"""

In [ ]:
import pandas as pd


path = '../data/recipes.zip'

recipes = pd.read_json(path, compression='infer', lines = True) 
recipes

### 1.2 A simple recipe recommender

Let's build a recipe recommender: given a list of basic ingredients, find a recipe that uses all those ingredients.

Here is the list of ingredients that can be asked for:

```
['salt', 'pepper', 'oregano', 'sage', 'parsley',
 'rosemary', 'tarragon', 'thyme', 'paprika', 'cumin']
```

**Hint:** Build a new column for each of the ingredients that indicates whether that ingredient is in the recipe.

#### Example

```
recommend_ingredients(["parsley", "paprika", "tarragon"], df)

result: 
# The rows where these 3 ingredients are in the recipe
[2069, 74964, 93768, 113926, 137686, 140530, 158475, 158486, 163175, 165243]
```

In [ ]:
# exercise 1.2
import re #Using a custom library to remove ... \n4, \n3, \n2, \n1, \n ... from a string
from ast import literal_eval


searchedlistIngredients = ["parsley", "paprika", "tarragon"]


data = (
    recipes.ingredients.apply(lambda row: re.sub(r'\n[0-9]?', ',', row)) #Using regex from 0-9. Anything match in the row is replaced by ','
    .apply(str)
    .apply(lambda stringofingredients: stringofingredients.lower()) #Transforming all characters to lower 
    ) 


recipes['ingredients']=data #repopulate the ingredient column with the updated string format

for ingredient in searchedlistIngredients:
    recipes[ingredient]= (recipes.ingredients.apply(lambda text: text.find(ingredient)) #First we search if the ingredient is present as a substring via the method .find(). 
                                                                                          #This returns -1 if False, or the first index of the substring if True
                            .apply(lambda integ: False if integ<0 else True) #The returns from the .find() method are converted to True or False
                         )
                         

recipes['result'] = True #Creation of a new column Result

for ingredient in searchedlistIngredients:  # The result column is updated with the Boolean product of the Boolean ingredient columns 
    recipes['result'] = recipes[ingredient]*recipes['result']

#We filter on the column which result is True and print the corresponding Indexes
resultSearchIngredients = recipes.loc[(recipes['result']==True)]
print(f'The rows where these 3 ingredients are in the recipe : {resultSearchIngredients.index.tolist()}')
 

    

## Exercise 2: Movies!

Look at this [Movies Dataset](https://www.kaggle.com/rounakbanik/the-movies-dataset) on Kaggle. Download it and use it for this question. 

The dataset is made up of several tables. The tables have common columns (`id` and `movie_id`) around which you can use to merge and join tables.

Your task is to find **The best director** in terms of average ratings of his movies. This can be from the `ratings` or `ratings_small` table, or simply the vote average in the `metadata` table. The director can be found in the `cast` table.

**💪You will have to use all of your skills to get this done, between using groupbys and merging multiple tables together.**

In [52]:
# exercise 2
import pandas as pd
import numpy as np
import json
from ast import literal_eval
# Use Pandas to assign CSV to each of the DataFrames. Will make it easier to organize your data

credits_df = pd.read_csv('../data/archive/credits.csv') #uses id / int64
ratings_df = pd.read_csv('../data/archive/ratings.csv') #uses movieId / int64

#Function to retrieve the director name from the list of dictionaries
def searchdirector(xlist):
    xlist.replace("'", '"')  #replacing simple by double quote to be readable as json / dictionary
    xlist1 = literal_eval(xlist) #reading the list inside the string
    
    name = None
    for dic in xlist1:
        key = 'job'
        value = dic.get(key, 0)
        if value != 0:
            if value=='Director':
                name = str(dic['name'])
    return name

#the director name is added to a new column next to each movie IDs
credits_df['director']= credits_df['crew'].apply(searchdirector)
creditsWithDirectors_df = credits_df.loc[(credits_df['director']!=None)]
 

    
    
    
 





,cast,crew,id,director
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862,John Lasseter
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844,Joe Johnston
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602,Howard Deutch
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357,Forest Whitaker
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862,Charles Shyer
...,...,...,...,...
45471,"[{'cast_id': 0, 'character': '', 'credit_id': ...","[{'credit_id': '5894a97d925141426c00818c', 'de...",439050,Hamid Nematollah
45472,"[{'cast_id': 1002, 'character': 'Sister Angela...","[{'credit_id': '52fe4af1c3a36847f81e9b15', 'de...",111109,Lav Diaz
45473,"[{'cast_id': 6, 'character': 'Emily Shaw', 'cr...","[{'credit_id': '52fe4776c3a368484e0c8387', 'de...",67758,Mark L. Lester
45474,"[{'cast_id': 2, 'character': '', 'credit_id': ...","[{'credit_id': '533bccebc3a36844cf0011a7', 'de...",227506,Yakov Protazanov


In [63]:
#Calculation: Movie Rating = Sum Ratings for a movie / Number of votes for this movie
#1- Number of votes for this movie
votes_per_movie = ratings_df['movieId'].value_counts()
ratings_df['vote counts'] = votes_per_movie

#2- Sum Ratings for a movie
tot_rating_permovie = ratings_df.groupby(['movieId'])['rating'].agg(np.sum).reset_index()

#3- Movie Rating
tot_rating_permovie['rating']=tot_rating_permovie['rating']/votes_per_movie
clean_rating = tot_rating_permovie.dropna()
clean_rating

,movieId,rating
1,2,1.277951
2,3,1.888392
3,4,0.553172
4,5,15.762496
5,6,7.023594
...,...,...
45074,176183,0.013289
45079,176193,0.500000
45081,176197,0.001617
45100,176237,0.073171


In [75]:
   
#Merging tables to find the association Best Movie Rating <=> Director
merge_rating_directors_df = pd.merge(creditsWithDirectors_df, clean_rating, left_on='id', right_on='movieId')


highestRating = merge_rating_directors_df['rating'].agg(np.max)
highestRating

bestDirector = merge_rating_directors_df.loc[(merge_rating_directors_df['rating']==highestRating)]
print(f'the best director is {bestDirector.director}')

the best director is 933    Brian De Palma
Name: director, dtype: object
